<a href="https://colab.research.google.com/github/GGpark1/Deep_Learning_Study/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [33]:
#파일 import

http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [34]:
#df 구성

lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 192341


In [35]:
lines.head()

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,Bouge !
3,Hi.,Salut !
4,Hi.,Salut.


In [36]:
lines = lines[0:60000]

In [37]:
lines.sample(10)

,src,tar
24483,Is that mandatory?,Est-ce obligatoire ?
24617,It was very windy.,C'était très venteux.
37704,They believe in God.,Eux croient en Dieu.
25208,She never told me.,Elle ne me l'a jamais dit.
5764,Tell me more.,Conte-m'en davantage !
5265,I'm stubborn.,Je suis obstinée.
7567,I'll get lost.,Je me perdrai.
36752,Look under the seat.,Regarde sous le siège.
3011,I can dance.,Je sais danser.
24949,My new car is red.,Ma nouvelle voiture est rouge.


In [38]:
#디코딩 문장의 앞 뒤에 시작과 종료 표시

lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(10)

,src,tar
22611,Don't tell anyone.,\t Ne le dites à quiconque ! \n
42325,I want you to try it.,\t Je veux que tu l'essayes. \n
32571,We're not sure yet.,\t Nous n'en sommes pas encore sûrs. \n
39381,You have to help me.,\t Vous devez m'aider. \n
39502,You sure are pretty.,\t Tu es vraiment jolie. \n
31695,This is a big help.,\t Ceci est d'une grande aide. \n
37397,That's Tom's sister.,\t C'est la sœur de Tom. \n
13524,He's a good lad.,\t C'est un mec bien. \n
59866,Who're you calling now?,\t Qui es-tu en train d'appeler ? \n
58025,That's not my decision.,\t Ce n'est pas ma décision. \n


In [39]:
#문자 집합 구축

src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [40]:
#문자 집합 사이즈 정의

src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1

print(src_vocab_size)
print(tar_vocab_size)

80
105


In [41]:
#문자 집합 리스트화

src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

print(src_vocab[45:75])
print(tar_vocab[0:10])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['\t', '\n', ' ', '!', '"', '$', '%', '&', "'", '(']


In [42]:
#문자에 인덱스 번호 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

In [43]:
#훈련 데이터 정수 인코딩

encoder_input = []

#문장 1개
for line in lines.src:
    encoded_line = []
    # 각 줄에서 1개의 char
    for char in line:
        #각 char을 정수로 변환
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)

print(encoder_input[:5])

[[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]


In [44]:
#테스트 데이터 정수 인코딩

decoder_input = []
for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)

print(decoder_input[:5])

[[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2]]


In [45]:
#디코더 출력값과 비교할 실제값 
decoder_target = []
for line in lines.tar:
    timestep = 0
    encoded_line = []
    for char in line:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
    decoder_target.append(encoded_line)
print(decoder_target[:5])

[[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2]]


In [46]:
#패딩 처리를 위한 데이터 샘플 길이 확인

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

print(max_src_len)
print(max_tar_len)

23
76


In [47]:
#패딩

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [48]:
#학습을 위해 정수 인코딩을 원핫인코딩으로 전환

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input) 
decoder_target = to_categorical(decoder_target)

In [49]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [50]:
#교사 강요

encoder_inputs = Input(shape = (None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True) #return_seqeunces가 True인 경우 모든 시점의 은닉 상태가 첫번째 값으로 출력됨(encoder_outputs)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs) #각각 최종 은닉 상태, 마지막 시점의 은닉 상태(앞의 값과 같음), 마지막 시점의 셀 상태
encoder_states = [state_h, state_c] #디코더에 넘겨주기 위한 컨텍스트 벡터

In [51]:
#디코더 설계

decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

#디코더에 인코더의 컨텍스트 벡터를 넘겨줌
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [52]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 346s 457ms/step - loss: 0.7480 - val_loss: 0.6706
Epoch 2/40
750/750 [==============================] - 340s 453ms/step - loss: 0.4591 - val_loss: 0.5420
Epoch 3/40
750/750 [==============================] - 343s 458ms/step - loss: 0.3828 - val_loss: 0.4718
Epoch 4/40
750/750 [==============================] - 343s 457ms/step - loss: 0.3395 - val_loss: 0.4328
Epoch 5/40
750/750 [==============================] - 339s 452ms/step - loss: 0.3104 - val_loss: 0.4074
Epoch 6/40
750/750 [==============================] - 340s 453ms/step - loss: 0.2890 - val_loss: 0.3910
Epoch 7/40
750/750 [==============================] - 343s 457ms/step - loss: 0.2729 - val_loss: 0.3812
Epoch 8/40
750/750 [==============================] - 336s 448ms/step - loss: 0.2600 - val_loss: 0.3706
Epoch 9/40
750/750 [==============================] - 343s 458ms/step - loss: 0.2493 - val_loss: 0.3651
Epoch 10/40
750/750 [==============================] - 343s 457m

In [53]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [54]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [55]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [56]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [57]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Hi.
정답 문장: Salut ! 
번역 문장: Salut. 
-----------------------------------
입력 문장: I see.
정답 문장: Aha. 
번역 문장: Je suis devant. 
-----------------------------------
입력 문장: Hug me.
정답 문장: Serrez-moi dans vos bras ! 
번역 문장: Serrez-moi dans vos concances. 
-----------------------------------
입력 문장: Help me.
정답 문장: Aidez-moi. 
번역 문장: Aide-moi. 
-----------------------------------
입력 문장: I am sure.
정답 문장: Je suis sûr. 
번역 문장: Je suis tellement occupé. 
